# SemLog_Web - Main functionalities demostration

This notebook shows two main functions of semlog_web, that is  "Entity Search" & "Event Search" and how to store and use the generated local image datasets. 

In [5]:
import sys
import os
sys.path.append("..")
from semlog_mongo.semlog_mongo.mongo import MongoDB,download_images
from semlog_mongo.semlog_mongo.utils import event_search
from semlog_vis.semlog_vis.bounding_box import *
from semlog_vis.semlog_vis.image import *
from semlog_vis.semlog_vis.create_annotation import *
from image_path.image_path import *
import pprint

In [6]:
# Define path to save images
IMAGE_ROOT="./saved_images"
IMAGE_FOLDER="one_example_search"

## 1. Entity Search
Entity search can be used to search for multiple objects or classes.<br>
Think about a case that we need to train a Binary Classifier for Watermelon and Pizza. 
We need to retrieve all images contains at least one object from these two classes.<br>
By using Entity Search we can get the result with one function call.

In [7]:
# Define parameters for MongoDB
IP="127.0.0.1"
PORT=27017
DATABASE_COLLECTION_LIST=[
    ['Food','f_in_drawers_0'],
    ['Food','f_on_counter_0'],
]

# Define parameters for entity searching
class_id_list=['MPWatermelon','MPBreadPizza']
object_pattern='class'
object_logic='or'

m=MongoDB(DATABASE_COLLECTION_LIST,IP,PORT)

In [8]:
result=m.entity_search(class_id_list=class_id_list,
                       object_pattern=object_pattern,
                       object_logic=object_logic)

['uLk3ljS8PkGuvt8wAjJM1w', 'kFJAq2cWmUa8_HApDUko3w', 'pcxaxPfcuE-3WrMiCYDzug', '3-KXJIMgZkSc6nfGrqxggQ']
Length of result: 1054
Length of result: 918


 The result is stored in a pandas.Dataframe().<br>
 Let's check the returned data.

In [9]:
result.head()

,class,collection,database,document,file_id,object,size,type
0,MPWatermelon,f_in_drawers_0,Food,5d5fd88ddd3ffb245c003a4e,5d5fda23dd3ffb245c00a16b,3-KXJIMgZkSc6nfGrqxggQ,0.003494,Color
1,MPWatermelon,f_in_drawers_0,Food,5d5fd88ddd3ffb245c003a4e,5d5fda23dd3ffb245c00a16d,3-KXJIMgZkSc6nfGrqxggQ,0.003494,Mask
2,MPWatermelon,f_in_drawers_0,Food,5d5fd88ddd3ffb245c003a57,5d5fda24dd3ffb245c00a16f,3-KXJIMgZkSc6nfGrqxggQ,0.004877,Color
3,MPWatermelon,f_in_drawers_0,Food,5d5fd88ddd3ffb245c003a57,5d5fda24dd3ffb245c00a171,3-KXJIMgZkSc6nfGrqxggQ,0.004877,Mask
4,MPWatermelon,f_in_drawers_0,Food,5d5fd88ddd3ffb245c003a60,5d5fda24dd3ffb245c00a173,3-KXJIMgZkSc6nfGrqxggQ,0.007603,Color


 Result records all qualified images and objects exist on them.<br>
 Let's download these images.

In [10]:
download_images(IP,IMAGE_ROOT,IMAGE_FOLDER,result)

Enter downloading!
Enter collection: ('Food', 'f_in_drawers_0')
Length of images 1054
make folder: ./saved_images\one_example_search
Enter collection: ('Food', 'f_on_counter_0')
Length of images 918


Right now all the images are stored in ./saved_images/one_example_search.

<img src="demo_images/d1.png">

## 1.1 GET cropped objects for training a classifer

 To execuate cropping, we need the mapping from object to mask color.

In [11]:
object_color_dict=m.get_object_rgb_dict(class_id_list,'class')
pprint.pprint(object_color_dict)

{'3-KXJIMgZkSc6nfGrqxggQ': (0, 188, 108),
 'kFJAq2cWmUa8_HApDUko3w': (94, 101, 69),
 'pcxaxPfcuE-3WrMiCYDzug': (47, 27, 162),
 'uLk3ljS8PkGuvt8wAjJM1w': (235, 182, 179)}


 Let's download the cropped images to the same folder.

In [12]:
download_bounding_box(result,object_color_dict,IMAGE_ROOT,IMAGE_FOLDER)

Object id: uLk3ljS8PkGuvt8wAjJM1w, rgb_color: (235, 182, 179)
Number of images: 225
Object id: kFJAq2cWmUa8_HApDUko3w, rgb_color: (94, 101, 69)
Number of images: 257
Object id: pcxaxPfcuE-3WrMiCYDzug, rgb_color: (47, 27, 162)
Number of images: 230
Object id: 3-KXJIMgZkSc6nfGrqxggQ, rgb_color: (0, 188, 108)
Number of images: 274


You can check the new images in the same folder.

<img src="demo_images/d3.png">

In [13]:
# (Optional) Resize all cropped images to a desired shape
image_dir=scan_bounding_box_images(IMAGE_ROOT,IMAGE_FOLDER,unnest=True)
resize_all_images(image_dir,width=80,height=80,resize_type='cut')

Enter resizing image.
Enter resizing. 80


All images are resized to 80x80 resolution.
<img src="demo_images/d4.png">

You can also pad the images to a desired resolution, the six padding types are from cv2.copyMakeBorder()

In [14]:
pad_all_images(image_dir,width=100,height=60,pad_type=cv2.BORDER_REFLECT)

Images are padded with the given pad_type.
<img src="demo_images/d5.png">

## 1.2 Get annotation of bounding boxes for training object detection models

In [15]:
# (Optional) resize all images to a desire shape before annotation, so that the bounding box coordinates can be calculated right.
image_dir=scan_images(IMAGE_ROOT,IMAGE_FOLDER,['Color','Mask'],unnest=True)
resize_all_images(image_dir,width=300,height=300,resize_type='cut')

# You can also scale images with a give ratio
# scale_all_images(image_dir,ratio=1.2)

Enter resizing image.
Enter resizing. 300


Images are rezized.
<img src="demo_images/d6.png">

In [16]:
# We still need the object_rgb_dict
df=calculate_bounding_box(result,object_color_dict,IMAGE_ROOT,IMAGE_FOLDER)

Object id: uLk3ljS8PkGuvt8wAjJM1w, rgb_color: (235, 182, 179)
Number of images: 225
Object id: kFJAq2cWmUa8_HApDUko3w, rgb_color: (94, 101, 69)
Number of images: 257
Object id: pcxaxPfcuE-3WrMiCYDzug, rgb_color: (47, 27, 162)
Number of images: 230
Object id: 3-KXJIMgZkSc6nfGrqxggQ, rgb_color: (0, 188, 108)
Number of images: 274


In [19]:
df=df[df['type']=='Color']
df1=df[['class','file_id','wmin','wmax','hmin','hmax']]
df1.head()

,class,file_id,wmin,wmax,hmin,hmax
0,MPWatermelon,5d5fda23dd3ffb245c00a16b,195,217,140,162
2,MPWatermelon,5d5fda24dd3ffb245c00a16f,210,234,99,124
4,MPWatermelon,5d5fda24dd3ffb245c00a173,220,247,48,79
6,MPWatermelon,5d5fda25dd3ffb245c00a177,226,257,1,37
8,MPWatermelon,5d5fda29dd3ffb245c00a18f,235,267,11,36


In [20]:
# Let's save the annotations
annotaion_path=os.path.join(IMAGE_ROOT,IMAGE_FOLDER,'annotation.txt')
class_mapping_path=os.path.join(IMAGE_ROOT,IMAGE_FOLDER,'class_mapping.txt')
convert_df_to_annotation(df,annotaion_path,class_mapping_path)

Now we have got both the images and annotations, you can copy the "Color" folder and "annotation.txt" to train a detection model now!

<img src="demo_images/d2.png">

# 2. Event Search
Event Search is used to search for exact images depending on conditions.

In [21]:
input_sentences=[['Food','f_in_drawers_0',3,'ViewReplicateDummy']]
df=event_search(IP,input_sentences)

In [22]:
# You can download these images as we did before
print(df.head())

       collection database                   file_id    type
0  f_in_drawers_0     Food  5d5fda33dd3ffb245c00a1c7   Color
1  f_in_drawers_0     Food  5d5fda33dd3ffb245c00a1c8   Depth
2  f_in_drawers_0     Food  5d5fda33dd3ffb245c00a1c9    Mask
3  f_in_drawers_0     Food  5d5fda33dd3ffb245c00a1ca  Normal
